# Join surrogate classes that are under the same node



In [1]:
import numpy as np
import os
from shutil import copyfile, copytree
from tqdm import tqdm

import sys
sys.path.append('../../python_scripts')
from utils import read_images_stl10 as read_images
from torchvision import transforms

from PIL import Image
from matplotlib import pyplot as plt

import shutil
from os.path import join

sys.path.insert(0, './../scikit_learn')
sys.path.insert(0, './../')
from utils_clust import normalizing_samples_L2, loading_images, searching_similar_images

/home/eric/.local/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# To load pairs of children and...
iteration_nb = '001_retrieval'
childrens_nb = '001'
path_pairs_out = './less_collisions/image_pairs' + childrens_nb
path_pairs_out_larger_clusters = './less_collisions/image_pairs_mixed_nodes' + childrens_nb

# other paths
path_target_dset_s_cl = './less_collisions/clusters/dset' + iteration_nb + '_short_cl/'
path_target_dset_l_cl = './less_collisions/clusters/dset' + iteration_nb + '_large_cl/'
path_new_classes = './less_collisions/clusters/new_classes_' + iteration_nb + '/'
path_target_dset_single = './less_collisions/clusters/dset' + iteration_nb + '_clusters_from_single_images'
path_single_old_classes = './less_collisions/clusters/old_classes_' + iteration_nb + '/'


# source images
# 1st iteration path
#images_path = '../../surrogate_dataset/classes_folder_16000_set/'
images_path = '../../surrogate_dataset/training_set_100000/'
# 2nd iteration path
#images_path = './images_2nd_iteration/'

# 3rd iteration path
#images_path = './images_3rd_iteration/'

In [3]:
short_clusters = []
larger_clusters = []

if os.path.exists(path_pairs_out  + '.npy'):
    short_clusters = np.load(path_pairs_out +'.npy')

if os.path.exists(path_pairs_out_larger_clusters  + '.npy'):
    larger_clusters = np.load(path_pairs_out_larger_clusters + '.npy')

In [4]:
print 'Arrays loaded stats: '
print "children_array:",
print len(short_clusters), type(short_clusters), short_clusters.shape

#print "sub_child_int_array:",
#print len(sub_child_int_array)

print "sub_child_mixed_array:",
print len(larger_clusters), type(larger_clusters), larger_clusters.shape

Arrays loaded stats: 
children_array: 2317 <type 'numpy.ndarray'> (2317, 2)
sub_child_mixed_array: 361 <type 'numpy.ndarray'> (361,)


## Build first the large clusters

In [5]:
# We need a set of all the images selected for clustering. 
# From there we will remove each image when it is added to a cluster.

samples_set = set()
for cluster_i in larger_clusters:
    for sample_i in cluster_i:
        samples_set.add(sample_i)
        
print "Larger cplusters introduced. Length: ",
print len(samples_set)
        
# I introduce the samples from the simple clusters as well

for cluster_i in short_clusters:
    for sample_i in cluster_i:
        samples_set.add(sample_i)
        
print "Short clusters (pairs) introduced. Length: ",
print len(samples_set)

Larger cplusters introduced. Length:  979
Short clusters (pairs) introduced. Length:  4951


In [6]:
# we need a set with all the 100k images from the dataset
samples_full = os.listdir(images_path)
samples_full_set = set([int(sample_i[:-4]) for sample_i in samples_full])

print len(samples_full), len(samples_full_set), type(samples_full_set)

100000 100000 <type 'set'>


In [7]:
# first we invert the larger_cluster array. This is to start clustering the larger groups.
# This is needed because the tree structure made clusters that are inside other clusters...

larger_clusters = larger_clusters[::-1]

# Note that we do not need to do that with the simple clusters

In [8]:
cluster_number = 0    # variable to count clusters
for cluster_i in larger_clusters:
    nb_img = len(cluster_i)
    num = 1
    temp = 0    # variable to count clusters
    for sample_i in cluster_i:       
        if sample_i in samples_set:
            temp +=1    # variable to count clusters
            
            ## defining paths 
            if not os.path.exists(join(path_target_dset_l_cl, 'cl_l_' + str(cluster_number).zfill(6))):
                os.makedirs(join(path_target_dset_l_cl, 'cl_l_' + str(cluster_number).zfill(6)))
                # 'cl_l' stands for clusters large
            
            src_path = join(images_path, str(sample_i).zfill(6) + '.png')
            dst_path = join(path_target_dset_l_cl, 'cl_l_' + str(cluster_number).zfill(6), str(sample_i).zfill(6) + '.png')
            
            ## moving files to the new clusterred dataset
            shutil.copyfile(src_path, dst_path)
            
            #### ploting...
            #image = Image.open(src_path)
            #plt.subplot(1,nb_img, num)
            #plt.imshow(np.asarray(image))
            #num += 1
            
            ### removing processed samples
            samples_set.remove(sample_i)
            samples_full_set.remove(sample_i)
    if temp > 0:
        cluster_number += 1   # variable to count clusters
    plt.show()
print "Number of larger clusters:",
print cluster_number

Number of larger clusters: 294


In [9]:
print "Set length after processing the large clusters: ",
print len(samples_set)

print "Set length of the full set after processing the large clusters: ",
print len(samples_full_set)

Set length after processing the large clusters:  3972
Set length of the full set after processing the large clusters:  99021


In [10]:
cluster_number = 0    # variable to count clusters
for cluster_i in short_clusters:
    nb_img = len(cluster_i)
    num = 1
    temp = 0    # variable to count clusters
    for sample_i in cluster_i:       
        if sample_i in samples_set:
            temp +=1    # variable to count clusters
            
            ## defining paths 
            if not os.path.exists(join(path_target_dset_s_cl, 'cl_s_' + str(cluster_number).zfill(6))):
                os.makedirs(join(path_target_dset_s_cl, 'cl_s_' + str(cluster_number).zfill(6)))
                # 'cl_l' stands for clusters large
            
            src_path = join(images_path, str(sample_i).zfill(6) + '.png')
            dst_path = join(path_target_dset_s_cl, 'cl_s_' + str(cluster_number).zfill(6), str(sample_i).zfill(6) + '.png')
            
            ## moving files to the new clusterred dataset
            shutil.copyfile(src_path, dst_path)
            
            ### ploting...
            #image = Image.open(images_path + str(sample_i).zfill(6) + '.png')
            #plt.subplot(1,nb_img, num)
            #plt.imshow(np.asarray(image))
            #num += 1.
            
            ### removing processed samples
            samples_set.remove(sample_i)
            samples_full_set.remove(sample_i)
    if temp > 0:
        cluster_number += 1   # variable to count clusters
    plt.show()
print "Number of smaller clusters:",
print cluster_number

Number of smaller clusters: 1986


In [12]:
print "Set length after processing the short clusters: ",
print len(samples_set)

print "Set length of the full set after processing the large clusters: ",
print len(samples_full_set)

Set length after processing the short clusters:  0
Set length of the full set after processing the large clusters:  95049


# Introduce "single cluster" (the images not clustered) to the dataset

In [13]:
# first we extract the images that we need to reach the full dataset, in this case 8000
max_classes = 8000
nb_new_classes = max_classes - (len(os.listdir(path_target_dset_l_cl)) + len(os.listdir(path_target_dset_s_cl)))

print "Checking numbers..."
print nb_new_classes

Checking numbers...
5720


In [14]:
# I will get the imaages from the unsup set
single_samples = [image_i for image_i in list(samples_full_set)[:nb_new_classes]]

# Save the images in a folder 

if not os.path.exists(path_new_classes):
    os.makedirs(path_new_classes)           

for idx in tqdm(single_samples):
    path = os.path.join(path_new_classes, str(idx).zfill(6))
    image = Image.open(images_path + str(idx).zfill(6) + '.png')
    samples_full_set.remove(idx)
    image.save(path + '.png') 

100%|██████████| 5720/5720 [00:11<00:00, 516.47it/s]


In [15]:
print "Set length after processing the short clusters: ",
print len(samples_set)

print "Set length of the full set after processing the large clusters: ",
print len(samples_full_set)

Set length after processing the short clusters:  0
Set length of the full set after processing the large clusters:  89329


## Searching the nearest images from unlabeled images - Single images

In [16]:
single_images = os.listdir(path_new_classes)
single_images.sort()

In [17]:
# We need the features to do the query search

# paths
features_path = './less_collisions/features_maxpool_allConv.hdf5'

# load images
samples = loading_images(features_path)
# normalize images
samples_L2 = normalizing_samples_L2(samples)

Dimension of the features:3440
Number of samples:100000


In [18]:
query_mtx = np.zeros([len(single_images), len(samples_L2[0])])
query_names = []
for num, sample_i in enumerate(single_images):
    query = samples_L2[int(sample_i[:-4])]
    query_names.append(sample_i)
    query_mtx[num] = query
    
# compute cosine similarity
sim = np.dot(query_mtx, samples_L2.T)

# sort ranking
ranks = np.argsort(sim, axis=1)[:,::-1]

In [19]:
cluster_size = 8

for num, sample_i in enumerate(query_names):
    cluster_i = []
    cluster_i.append(sample_i)
    
    instance_num = 1
    while len(cluster_i)<cluster_size:
        image_nb = ranks[num][instance_num]
        image_name = str(image_nb).zfill(6) + '.png'
        if image_nb in samples_full_set:
            cluster_i.append(image_name)
            samples_full_set.remove(image_nb)
        
        instance_num += 1
    
    # move the images fom the cluster_i to the new folder
    
    src_path = join(path_new_classes, cluster_i[0])
    dst_folder =  join(path_target_dset_single, cluster_i[0][:-4])

    if not os.path.exists(dst_folder):
        os.makedirs(dst_folder)
    
    dst_path = join(dst_folder, cluster_i[0])
    shutil.copyfile(src_path, dst_path)
    
    for num_cl, img_i in enumerate(cluster_i[1:]):
        src_path = join(images_path, cluster_i[num_cl+1])
        dst_path = join(dst_folder, img_i)
        shutil.copyfile(src_path, dst_path)

In [20]:
print "Set length after processing the short clusters: ",
print len(samples_set)

print "Set length of the full set after processing the large clusters: ",
print len(samples_full_set)

Set length after processing the short clusters:  0
Set length of the full set after processing the large clusters:  49289


In [ ]:
# after that i should do the same with the other clusters with 2 images each...

In [51]:
print sim.shape
print sim[:5]
print sim[-5:]
print ''
print max(sim)
print min(sim)
sim_ord = np.argsort(sim)
print ''
print sim_ord[:5]
print sim_ord[-5:]

(100000,)
[1.         0.86025035 0.85134862 0.83130919 0.81856119]
[0.82950443 0.83131215 0.82241898 0.85949542 0.82282796]

1.0000000000000022
0.5091351520829155

[62082 69907 28668 90896 69681]
[17744 67069 32344 88365     0]


In [ ]:
...
# finally we move all the single classes to a folder inside the "dataset00?" folder
list_files = os.listdir(path_new_classes)
for sample_i in tqdm(list_files):
    src_path = join(path_new_classes, sample_i)
    dst_folder =  join(path_target_dset, sample_i[:-4])
    
    if not os.path.exists(dst_folder):
        os.makedirs(dst_folder)
    
    dst_path = join(dst_folder, sample_i)
    shutil.copyfile(src_path, dst_path)

## Searching the nearest images from unlabeled images - Simple clusters

#### This one is the same but changing the last cell for some code to search for near images in the 50000 samples left. The aim is get larger clusters for the single images


In [22]:
# paths
features_path = './features_maxpool_allConv.hdf5'

In [23]:
# load images
samples = loading_images(features_path)
# normalize images
samples_L2 = normalizing_samples_L2(samples)

Dimension of the features:1792
Number of samples:100000


In [13]:
current_images = len(os.listdir(images_path))
current_images

16315

In [14]:
unlab_set = read_images('../../data/stl10_binary/unlabeled_X.bin')
np.random.seed(42)

# indexes drawn in a set to avoid duplicates
indexes = set()
while len(indexes) < (current_images + nb_new_classes): # we do that to get only the last ones
    indexes.add(np.random.randint(unlab_set.shape[0]))
    
# Save the images in a folder 
toPill = transforms.Compose([transforms.ToPILImage()])

if not os.path.exists(path_new_classes):
    os.makedirs(path_new_classes)           

for num, idx in tqdm(enumerate(list(indexes)[-nb_new_classes:])):
    path = os.path.join(path_new_classes, str(num + current_images).zfill(len(str(max_classes))))
    image = toPill(unlab_set[idx])
    image.save(path + '.png') 


299it [00:00, 687.95it/s]


In [15]:
# now we copy the images that we did not use during the clustering in another folder
if not os.path.exists(path_single_old_classes):
    os.makedirs(path_single_old_classes)

for sample_i in tqdm(samples_full_set):
    src_path = join(images_path, str(sample_i).zfill(5) + '.png')
    dst_path = join(path_single_old_classes, str(sample_i).zfill(5) + '.png')
    shutil.copyfile(src_path, dst_path)

100%|██████████| 15116/15116 [00:00<00:00, 23543.60it/s]


In [16]:
# finally we move all the single classes to a folder inside the "dataset00?" folder
list_files = os.listdir(path_new_classes)
for sample_i in tqdm(list_files):
    src_path = join(path_new_classes, sample_i)
    dst_folder =  join(path_target_dset, sample_i[:-4])
    
    if not os.path.exists(dst_folder):
        os.makedirs(dst_folder)
    
    dst_path = join(dst_folder, sample_i)
    shutil.copyfile(src_path, dst_path)


list_files = os.listdir(path_single_old_classes)
for sample_i in tqdm(list_files):
    src_path = join(path_single_old_classes, sample_i)
    dst_folder =  join(path_target_dset, sample_i[:-4])
    
    if not os.path.exists(dst_folder):
        os.makedirs(dst_folder)

    dst_path = join(dst_folder, sample_i)
    shutil.copyfile(src_path, dst_path)

100%|██████████| 15116/15116 [00:01<00:00, 14203.86it/s]


### Now is time to compute the transformations from the new clustered dataset...